In [1]:
import os
import re
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
csv_folder_name = 'csv_data_hate'
total_samples = 3000

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(csv_folder_name)
regex = re.compile('([a-zA-Z0-9_-]*)_final\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(csv_folder_name, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path, index_col=0)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<20}{:<7}'.format(topic_name, row_count))


ageism_04           1600   
ageism_08           709    
ageism_12           537    
ageism_2020-02      117    
ageism_2021-02      237    
ageism_2021-04      691    
asianhate_04        2887   
asianhate_08        815    
asianhate_12        399    
asianhate_2020-02   538    
asianhate_2021-02   285    
asianhate_2021-04   323    
immigration_04      93     
immigration_08      144    
immigration_12      52     
immigration_2020-02 183    
immigration_2021-02 39     
immigration_2021-04 186    
mask_04             16     
mask_08             91     
mask_12             63     
mask_2020-02        2      
mask_2021-02        53     
mask_2021-04        117    
politics_04         4617   
politics_08         7589   
politics_12         2883   
politics_2020-02    2706   
politics_2021-02    576    
politics_2021-04    263    
vaccine_04          475    
vaccine_08          436    
vaccine_12          228    
vaccine_2020-02     94     
vaccine_2021-02     112    
vaccine_2021-04     

In [4]:
df_count['minimum'] = df_count['count'].apply(lambda x: x if x <= 20 else 20)
df_count['subtract'] = df_count['count'] - df_count['minimum']
min_sum = df_count['minimum'].sum()
df_count['proportion_count'] = df_count['subtract'].apply(lambda x: round((x/(total_count - min_sum))*(total_samples - min_sum)))
df_count['proportion_count'] += df_count['minimum']

if df_count['proportion_count'].sum() > total_samples:
    diff = df_count['proportion_count'].sum() - total_samples
    df_count.at[df_count.proportion_count.idxmax(), 'proportion_count'] -= diff
elif df_count['proportion_count'].sum() < total_samples:
    diff = total_samples - df_count['proportion_count'].sum()
    df_count.at[df_count.proportion_count.idxmax(), 'proportion_count'] += diff

df_count

,topic,count,proportion_count,minimum,subtract
0,ageism_04,1600,143,20,1580
1,ageism_08,709,74,20,689
2,ageism_12,537,60,20,517
3,ageism_2020-02,117,28,20,97
4,ageism_2021-02,237,37,20,217
5,ageism_2021-04,691,72,20,671
6,asianhate_04,2887,243,20,2867
7,asianhate_08,815,82,20,795
8,asianhate_12,399,49,20,379
9,asianhate_2020-02,538,60,20,518


In [ ]:
# df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))
# df_count['proportion_count'] = round(total_samples / len(df_count.index))
# df_count.proportion_count[df_count.topic=='mask_08'] += 1
# df_count

In [5]:
df_count.to_csv(csv_folder_name + '/sample_count.csv', index=False)
df_count['proportion_count'].sum()

3000

In [6]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'sample_path', 'body_text', 'image_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_topic.insert(loc=4, column='sample_path', value=np.nan)
    df_topic.insert(loc=len(df_topic.columns), column='image_text', value=np.nan)
    df_samples = df_samples.append(df_topic)

df_samples = df_samples.sample(frac=1, random_state=42).reset_index(drop=True)
df_samples.to_csv(csv_folder_name + '/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,politics_08,RacistInChief,1296147677466501121,./tweet_data/politics_08/RacistInChief/tweets3...,NaN,#Magats your #RacistInChief despises #MAGA vot...,NaN,lt es voter love voter total loser voter love ...
1,mask_2021-04,masksdontwork,1378756189941547008,./tweet_data/mask_2021-04/masksdontwork/tweets...,NaN,"Wow, Dr. Doom pretty much wrong about everythi...",NaN,new confirm case 30k 25k 20k 15k new confirm c...
2,politics_08,AmericaOrTrump,1290967777235423232,./tweet_data/politics_08/AmericaOrTrump/tweets...,NaN,@realDonaldTrump I didn’t realize that making ...,NaN,normal en
3,ageism_12,okboomer,1339459062354735105,./tweet_data/ageism_12/okboomer/tweets3/Epa2xT...,NaN,#Imperfection.\nNever say that you are sorry f...,NaN,live life rule pi yn sr aw se
4,politics_2020-02,kag,1227630503265595392,./tweet_data/politics_2020-02/kag/tweets2/EQlr...,NaN,G’mornafternoovening twittersphere! #KAG #LRA ...,NaN,incumbent voting new hampshire 2004 2012 vs ay...
...,...,...,...,...,...,...,...,...
2995,politics_08,TrumpChaos,1298442582297473024,./tweet_data/politics_08/TrumpChaos/tweets4/Eg...,NaN,@SRuhle Remember 45 was IMPEACHED. #TrumpChaos...,NaN,oe impeached al rast
2996,immigration_2021-04,illegalimmigrants,1380207148341342208,./tweet_data/immigration_2021-04/illegalimmigr...,NaN,@nytimes This is a massive transfer of #Wealth...,NaN,youre playing game
2997,mask_08,MasksOffAmerica,1298411304579788800,./tweet_data/mask_08/MasksOffAmerica/tweets4/E...,NaN,@jaze_ca @jrstonelive @abc7newsbayarea 😂😂😂😂 NO...,NaN,ts wnat virologist wear protect virus eel ve r...
2998,politics_04,LiarInChief,1254174180334952448,./tweet_data/politics_04/LiarInChief/tweets4/E...,NaN,@realDonaldTrump Real journalists are trained ...,NaN,lie big simple ps say eventually ss believe ad...


# Please only run cells below

## Copy selected samples and generate sample path

### Before running this part, please check that the topics you want to process is NOT in the annotation_data folder

In [7]:
import os
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

In [8]:
# df_count = pd.read_csv(csv_folder_name + '/sample_count.csv')
df_samples = pd.read_csv(csv_folder_name + '/sample_data.csv')
annot_data_path = './annot_data_hate'
Path(annot_data_path).mkdir(exist_ok=True)

In [9]:
def copy_image_set_path(img_path):
    image_name = img_path.split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(img_path, dst_path)
    return dst_path

In [10]:
df_samples['sample_path'] = df_samples['image_path'].apply(copy_image_set_path)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,politics_08,RacistInChief,1296147677466501121,./tweet_data/politics_08/RacistInChief/tweets3...,./annot_data_hate/EfzVXLnX0AIp4ZB.jpg,#Magats your #RacistInChief despises #MAGA vot...,NaN,lt es voter love voter total loser voter love ...
1,mask_2021-04,masksdontwork,1378756189941547008,./tweet_data/mask_2021-04/masksdontwork/tweets...,./annot_data_hate/EyJTcl3WEAIGS0A.jpg,"Wow, Dr. Doom pretty much wrong about everythi...",NaN,new confirm case 30k 25k 20k 15k new confirm c...
2,politics_08,AmericaOrTrump,1290967777235423232,./tweet_data/politics_08/AmericaOrTrump/tweets...,./annot_data_hate/EepwWgJWoAA3pBS.jpg,@realDonaldTrump I didn’t realize that making ...,NaN,normal en
3,ageism_12,okboomer,1339459062354735105,./tweet_data/ageism_12/okboomer/tweets3/Epa2xT...,./annot_data_hate/Epa2xTmVEAA70b9.jpg,#Imperfection.\nNever say that you are sorry f...,NaN,live life rule pi yn sr aw se
4,politics_2020-02,kag,1227630503265595392,./tweet_data/politics_2020-02/kag/tweets2/EQlr...,./annot_data_hate/EQlrbqFVAAACJSY.jpg,G’mornafternoovening twittersphere! #KAG #LRA ...,NaN,incumbent voting new hampshire 2004 2012 vs ay...
...,...,...,...,...,...,...,...,...
2995,politics_08,TrumpChaos,1298442582297473024,./tweet_data/politics_08/TrumpChaos/tweets4/Eg...,./annot_data_hate/EgT-h2FU4AAJZwR.jpg,@SRuhle Remember 45 was IMPEACHED. #TrumpChaos...,NaN,oe impeached al rast
2996,immigration_2021-04,illegalimmigrants,1380207148341342208,./tweet_data/immigration_2021-04/illegalimmigr...,./annot_data_hate/Eyd7FWXXIAM3iF3.jpg,@nytimes This is a massive transfer of #Wealth...,NaN,youre playing game
2997,mask_08,MasksOffAmerica,1298411304579788800,./tweet_data/mask_08/MasksOffAmerica/tweets4/E...,./annot_data_hate/EgTiMZ7UYAMBLAT.png,@jaze_ca @jrstonelive @abc7newsbayarea 😂😂😂😂 NO...,NaN,ts wnat virologist wear protect virus eel ve r...
2998,politics_04,LiarInChief,1254174180334952448,./tweet_data/politics_04/LiarInChief/tweets4/E...,./annot_data_hate/EWe3qpjUMAAnV2P.jpg,@realDonaldTrump Real journalists are trained ...,NaN,lie big simple ps say eventually ss believe ad...


In [11]:
df_samples.to_csv(csv_folder_name + '/sample_data.csv', index=False)

### Old code below, don't run

In [3]:
def get_sample_path_and_copy(df, topic, annot_data_path):
    # if the sample_path already exists
    if not pd.isna(df['sample_path']):
        return df['sample_path']

    # if topic is not the selected topic
    if pd.isna(df['sample_path']) and df['topic'] != topic:
        return np.nan
    
    image_name = df['image_path'].split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(df['image_path'], dst_path)
    return dst_path

In [4]:
sample_folder = './annotation_data'
Path(sample_folder).mkdir(exist_ok=True)

for _, row in df_count.iterrows():
    topic_name = row['topic']
    topic_data_path = os.path.join('./', 'data_' + topic_name)
    annot_data_path = os.path.join(sample_folder, topic_name)
    # if data_{topic_name} folder doesn't exist
    if not Path(topic_data_path).is_dir():
        print('{} not exists; pass this topic'.format(topic_data_path))
        continue

    # if the sampled data for this topic already exists
    if Path(annot_data_path).is_dir():
        print('{} already exist; pass this topic'.format(annot_data_path))
        continue

    Path(annot_data_path).mkdir()

    df_samples['sample_path'] = df_samples.apply(get_sample_path_and_copy, axis=1, topic=topic_name, annot_data_path=annot_data_path)

    print('Topic \"{}\" processed'.format(topic_name))

Topic "Vaccine" processed
./data_AsianHate not exists; pass this topic
./data_Boomer not exists; pass this topic
./data_Political not exists; pass this topic
./data_Mask not exists; pass this topic
Topic "Immigration" processed


In [5]:
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,Immigration,refugees,1297606828461883394,./data_Immigration/refugees/tweets4/EgIGg6aXsA...,./annotation_data/Immigration/EgIGg6aXsAAX5-Y.jpg,Just one week left to get one of these hand em...,NaN,oe
1,Mask,maskup,1297309098669510656,./data_Mask/maskup/tweets4/EgD3vYTUcAE_JMC.jpg,NaN,🚧 #SFSO keeping our citizens updated on evacua...,NaN,er ha an ey st ge at Bra wy yo ae ae ay at oes...
2,Boomer,caresact,1291464418715459589,./data_Boomer/caresact/tweets1/Eew0B6PWAAYZPCX...,NaN,"Arguably, this one segment from the #CARESAct ...",NaN,aS is he
3,Mask,NoMasks,1290561571312328706,./data_Mask/NoMasks/tweets1/Eej-523WsAAJa6h.jpg,NaN,@RupertaMargate @FrancaisFarage @John35542712 ...,NaN,Ser ah
4,Immigration,cdnpoli,1297693936782213123,./data_Immigration/cdnpoli/tweets4/EgJVvpkUMAA...,./annotation_data/Immigration/EgJVvpkUMAALB8Z.png,"You beat me to it, lol .. #cpc #cpcldr #cdnpol...",NaN,You organize two car funeral procession iS
...,...,...,...,...,...,...,...,...
2395,Mask,NoMasks,1291460379638484994,./data_Mask/NoMasks/tweets1/EewwXv5WkAEb91v.jpg,NaN,What's this...a new wave of virus???\n\n#KBF #...,NaN,Let me introduce you to our next problem Seen ...
2396,Boomer,trumpliesamericansdie,1290437319611342851,./data_Boomer/trumpliesamericansdie/tweets1/Ee...,NaN,@realDonaldTrump 1st-To be so proud of a large...,NaN,reed la HI WA ha ant aa Trump to draw out at l...
2397,Boomer,caresact,1293904775055052801,./data_Boomer/caresact/tweets2/EfTfiMrWsAA35Y4...,NaN,Ready to re-open? Use your #CARESAct Funding t...,NaN,YOUR GO HERE Oe WEVE GOT vol
2398,Political,Trump2020,1291952240534728704,./data_Political/Trump2020/tweets2/Ee3vtmRX0AA...,NaN,@HowDoYouMakeAU2 @Jenny06980536 @TerriSm600279...,NaN,al LEAVE NEVER LEAVE
